In [1]:
versioninfo()

Julia Version 1.0.3
Commit 099e826241 (2018-12-18 01:34 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-6.0.0 (ORCJIT, skylake)


In [2]:
using DifferentialEquations, ParameterizedFunctions, PyPlot

In [3]:
g31 = @ode_def g3oneloop begin
    # note all couplings are for squared, i.e. "g1"==g1^2, except l = lambda
    # note g1^2 = 5/3*gy^2
    dg3 = g3^2/(4*pi)^2*(-7)
end
;

In [4]:
obsmt = 173.34
g30 = 1.1666
vev = 246
yt0 = 0.93690
;

In [5]:
running = g31
prob = ODEProblem(running, [g30^2], (2*log(obsmt),2*log(4.2)))
sol = solve(prob,reltol=1e-6,abstol=1e-6)
sqrt.([sol[1] sol[end]])

1×2 Array{Float64,2}:
 1.1666  1.57142

In [6]:
b0 = 11-2/3*5;
mb0 = 4.2*(sol[end][1]/sol[1][1])^(-4/b0)
yb0 = sqrt(2)*mb0/vev
[mb0 yb0]

1×2 Array{Float64,2}:
 3.07789  0.0176943

In [7]:
mtau0 = 1.77686
ytau0 = sqrt(2)*mtau0/vev

0.010214876058692106

In [8]:
b1 = @ode_def oneloop begin
    # note all couplings are for squared, i.e. "g1"==g1^2, except l = lambda
    # note g1^2 = 5/3*gy^2
    dg3 = g3^2/(4*pi)^2*(-7)
    dyt = yt/(4*pi)^2*(9/2*yt +3/2*yb +ytau -8*g3)
    dyb = yb/(4*pi)^2*(3/2*yt +9/2*yb +ytau -8*g3)
    dytau = ytau/(4*pi)^2*(3*yt +3*yb +5/2*ytau)
end
;

In [9]:
running = b1
initial = [g30^2 ; yt0^2 ; yb0^2 ; ytau0^2]
prob = ODEProblem(running, initial, (2*log(obsmt),2*log(3000)))
sol = solve(prob,reltol=1e-6,abstol=1e-6)
temp = sqrt.(sol(2*log(3000)))
(yt, yb, ytau) = temp[2:end]

3-element Array{Float64,1}:
 0.8435139959363097  
 0.015266079415271215
 0.010659898336397102

In [10]:
(mt, mb, mtau) = [yt, yb, ytau]*vev/sqrt(2)

3-element Array{Float64,1}:
 146.72779877187233  
   2.6555080760609773
   1.8542708545046944